In [1]:
#Code written by Marvin M. Vargas Flores in July 2019 for Accenture DTIS.
#Duckiebot Line Follower 
#Image processing mockup
#
#
#
#
#Include opencv and numpy modules 
import cv2
import numpy as np

#define threshold values of color filter
low_green = np.array([0,0,180])
up_green = np.array([120,255,255])
#Scale of the image, where 1 is 640*480
scale = 1
#Variable for the biggest contour area and the index of the contour in the contour array
bc = [0,0]
#Select webcam no.* to take the video
cap = cv2.VideoCapture(1)
#Declare the font to write on the image
font = cv2.FONT_HERSHEY_SIMPLEX
#Set the width of the taken frame
cap.set(3,scale*640)
#Set the height of the taken frame
cap.set(4,scale*480)


while True:
    #Take the image from the webcam
    ret,frame = cap.read()
    
    frame[0:20,0:640] = [0,0,0]
    frame[460:480,0:640] = [0,0,0]
    
    #Turn image from BGR to HSV space color
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    #Create color mask 
    mask = cv2.inRange(hsv,low_green,up_green)
    
    #Apply color filter to original image
    res = cv2.bitwise_and(hsv,hsv,mask= mask)
    
    #Turn image to grayscale
    bgr = cv2.cvtColor(res,cv2.COLOR_HSV2BGR)
    gray = cv2.cvtColor(bgr,cv2.COLOR_BGR2GRAY)
    
    #Turn mask to binary
    mret,mt = cv2.threshold(gray,125,255,cv2.THRESH_BINARY)
    
    #Apply a blur filter
    #blur = cv2.blur(mt,(5,5))
    
    #Use morphological dilation to reduce noise in the line
    kernel = np.ones((12,12),np.uint8)
    ker_er = np.ones((15,15),np.uint8)
    #erode = cv2.erode(mt,ker_er,iterations = 1)
    dilate = cv2.dilate(mt,kernel,iterations= 1)
    close = cv2.morphologyEx(dilate,cv2.MORPH_CLOSE,kernel)
    
    #Obtain the canny edges
    edge = cv2.Canny(close,30,200,True,3)
    
    #Do a final threshold of the canny edges to find contours
    fret,fthresh = cv2.threshold(edge,100,255,cv2.THRESH_BINARY)
    
    #Read the contours 
    contours,hierarchy = cv2.findContours(fthresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    contoured = frame.copy()
    
    #Declare counter variable
    i = 0
    #Check if any contour is found
    if len(contours):
        #Draw the contours in the image
        contoured = cv2.drawContours(frame,contours,-1,(0,255,0),3)
        #Create for loop to run over all the countours
        for c in contours:
            #Check if the actual contour area is the biggest
            act = cv2.contourArea(c)
            if act>bc[0]:
                #Actual contour area is new biggest
                bc = [act,i]
            #Add 1 to the i variable to keep counting the index
            i+=1
        #Draw in red the biggest contour in the image
        contoured = cv2.drawContours(frame,[contours[bc[1]]],-1,(0,0,255),3)    
        
        #Find the moments of the biggest contour
        M = cv2.moments(contours[bc[1]])

        #Find centroid of the contour
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])

        #Draw point that represents the centroid 
        cv2.circle(contoured,(cx,cy),10,(255,0,0),-1)
        #Draws the real center of the robot
        cv2.circle(contoured,(320,cy),10,(0,0,255),-1)
        
        #Calculate the twist message
        if(cx<=320):
            dx = 320-cx
            ang = dx/320
            linx = 1-ang
        else:
            dx = 320-cx
            ang = dx/320
            linx = 1-abs(ang)
    
        #Write the twist message in the image
        cv2.putText(contoured,'({},0,0),(0,0,{})'.format(linx,ang),(0,30), font, 1, (200,255,155), 2, cv2.LINE_AA)
        
    #Contours is the original image with the contours marked on green
    cv2.imshow('contours',contoured)
    #Final_thresh is the binary image where we look for the contours
    cv2.imshow('Final_thresh',fthresh)
    #Dilation is the binary image after processing the color filtered image
    cv2.imshow('Morphological ',close)
    #Mask is the color filtered image
    cv2.imshow('Mask',mt)
    bc=[0,0]
    if cv2.waitKey(1) & 0xFF == ord('q') :
        break
        
cap.release()
cv2.destroyAllWindows()

TypeError: 'NoneType' object does not support item assignment